# Langchain Démo

In [1]:
from dotenv import load_dotenv

# Charge les variables d'environnement depuis '.env'
load_dotenv()

True

## Chargement du document

`pip install pdfminer`
`pip install pdfminer.six`

In [2]:
from langchain_community.document_loaders import PDFMinerLoader


In [3]:
loader = PDFMinerLoader("uploads/code_penal.pdf")
code_penal = loader.load()

Tout le code pénal : 

In [4]:
code_penal

[Document(page_content='Code pénal\n\nPartie législative\n\nLivre Ier : Dispositions générales\n\nTitre Ier : De la loi pénale\n\nChapitre Ier : Des principes généraux\n\nArticle 111-1\n\nLes infractions pénales sont classées, suivant leur gravité, en crimes, délits et contraventions.\n\nArticle 111-2\n\nLa loi détermine les crimes et délits et fixe les peines applicables à leurs auteurs.\n\nLe règlement détermine les contraventions et fixe, dans les limites et selon les distinctions établies par la loi,\nles peines applicables aux contrevenants.\n\nArticle 111-3\n\nNul ne peut être puni pour un crime ou pour un délit dont les éléments ne sont pas définis par la loi, ou pour\nune contravention dont les éléments ne sont pas définis par le règlement.\n\nNul ne peut être puni d\'une peine qui n\'est pas prévue par la loi, si l\'infraction est un crime ou un délit, ou par\nle règlement, si l\'infraction est une contravention.\n\nArticle 111-4\n\nLa loi pénale est d\'interprétation stricte.

In [5]:
len(code_penal)

1

## Découpage du document

In [6]:
from langchain.text_splitter import CharacterTextSplitter

In [7]:
article_splitter = CharacterTextSplitter(
    separator = "\nArticle",
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
    keep_separator=True
)

In [8]:
articles = article_splitter.split_documents(code_penal)

Created a chunk of size 136, which is longer than the specified 100
Created a chunk of size 114, which is longer than the specified 100
Created a chunk of size 260, which is longer than the specified 100
Created a chunk of size 371, which is longer than the specified 100
Created a chunk of size 424, which is longer than the specified 100
Created a chunk of size 448, which is longer than the specified 100
Created a chunk of size 796, which is longer than the specified 100
Created a chunk of size 517, which is longer than the specified 100
Created a chunk of size 391, which is longer than the specified 100
Created a chunk of size 238, which is longer than the specified 100
Created a chunk of size 255, which is longer than the specified 100
Created a chunk of size 352, which is longer than the specified 100
Created a chunk of size 550, which is longer than the specified 100
Created a chunk of size 428, which is longer than the specified 100
Created a chunk of size 650, which is longer tha

Nombres approximatif d'articles :

In [9]:
len(articles)

1229

Apercu d'un article :

In [10]:
articles[1].page_content

'Article 111-1\n\nLes infractions pénales sont classées, suivant leur gravité, en crimes, délits et contraventions.'

In [11]:
articles[13].page_content

"Article 113-3\n\nCode pénal - Dernière modification le 01 avril 2023 - Document généré le 15 avril 2023\n\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n\x0cLa loi pénale française est applicable aux infractions commises à bord des navires battant un pavillon\nfrançais, ou à l'encontre de tels navires ou des personnes se trouvant à bord, en quelque lieu qu'ils se\ntrouvent. Elle est seule applicable aux infractions commises à bord des navires de la marine nationale, ou à\nl'encontre de tels navires ou des personnes se trouvant à bord, en quelque lieu qu'ils se trouvent."

In [12]:
articles[-1].page_content

'Article R722-7\n\nLe deuxième alinéa de l\'article R. 131-36 est rédigé comme suit :\n\n" Lorsque le personnel de cette personne morale est régi par les dispositions applicables localement relatives\nà la représentation des salariés, l\'avis mentionné au premier alinéa est adressé au secrétaire du comité\n\nCode pénal - Dernière modification le 01 avril 2023 - Document généré le 15 avril 2023\n\n \n \n \n \n \n \n \n \n \n \n \n\x0cd\'entreprise ou, le cas échéant, au secrétaire du comité central d\'entreprise et, en l\'absence de tels comités,\naux délégués du personnel titulaires. "\n\nChapitre III : Adaptation du livre II\n\nChapitre IV : Adaptation du livre III\n\nChapitre V : Adaptation du livre IV\n\nChapitre VI : Adaptation du livre V\n\nChapitre VII : Adaptation du livre VI\n\nCode pénal - Dernière modification le 01 avril 2023 - Document généré le 15 avril 2023'

## Embedding du document

In [13]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

## Vectorisation des articles

`pip install chromadb`

In [14]:
from langchain_community.vectorstores import Chroma

In [15]:
vector_db = Chroma.from_documents(articles,embeddings_model) 

## Requête à la bdd vectoriel

In [16]:
query = "Prison à perpétuité"
docs = vector_db.similarity_search(query, k=4)
for doc in docs:
    print(doc.page_content)
    print("------")

Article 132-8

Lorsqu'une personne physique, déjà condamnée définitivement pour un crime ou pour un délit puni de dix
ans d'emprisonnement par la loi, commet un crime, le maximum de la peine de la réclusion criminelle ou de
la détention criminelle est la perpétuité si le maximum fixé par la loi pour ce crime est de vingt ou trente ans.
Le maximum de la peine est porté à trente ans de réclusion criminelle ou de détention criminelle si le crime
est puni de quinze ans.
------
Article 422-2

La peine privative de liberté encourue par l'auteur ou le complice d'un acte de terrorisme est réduite de
moitié si, ayant averti les autorités administratives ou judiciaires, il a permis de faire cesser les agissements
incriminés ou d'éviter que l'infraction n'entraîne mort d'homme ou infirmité permanente et d'identifier, le cas
échéant, les autres coupables. Lorsque la peine encourue est la réclusion criminelle à perpétuité, celle-ci est
ramenée à vingt ans de réclusion criminelle.
------
Article 221

## Questions / Réponses avec le code pénal

### Prompt Template

In [17]:
from langchain.prompts import PromptTemplate

In [18]:
QA_PROMPT = PromptTemplate(
    template="""Voici des articles extraits du code pénal français : {text}
À partir de ces articles répond à la question suivante : {question}""",
    input_variables=["text", "question"]
)

### Chargement du LLM

In [19]:
from langchain_openai import ChatOpenAI

In [20]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0)

### Création de la chaîne

In [21]:
retriever = vector_db.as_retriever()

# setup_and_retrieval = RunnableParallel(
#     {"context": retriever, "question": RunnablePassthrough()}
# )

In [22]:
retriever.invoke("Prison à perpétuité")

[Document(page_content="Article 132-8\n\nLorsqu'une personne physique, déjà condamnée définitivement pour un crime ou pour un délit puni de dix\nans d'emprisonnement par la loi, commet un crime, le maximum de la peine de la réclusion criminelle ou de\nla détention criminelle est la perpétuité si le maximum fixé par la loi pour ce crime est de vingt ou trente ans.\nLe maximum de la peine est porté à trente ans de réclusion criminelle ou de détention criminelle si le crime\nest puni de quinze ans.", metadata={'source': 'uploads/code_penal.pdf'}),
 Document(page_content="Article 422-2\n\nLa peine privative de liberté encourue par l'auteur ou le complice d'un acte de terrorisme est réduite de\nmoitié si, ayant averti les autorités administratives ou judiciaires, il a permis de faire cesser les agissements\nincriminés ou d'éviter que l'infraction n'entraîne mort d'homme ou infirmité permanente et d'identifier, le cas\néchéant, les autres coupables. Lorsque la peine encourue est la réclusion

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"text": retriever, "question": RunnablePassthrough()}
    | QA_PROMPT
    | llm
    | StrOutputParser()
)



## Utilisation

In [24]:
res = rag_chain.invoke("Quels sont les conditions qui pourraient entraîner une condamnation à perpétuité?")

In [25]:
res

"Les conditions qui pourraient entraîner une condamnation à perpétuité sont les suivantes :\n\n- Le meurtre commis sur un mineur de quinze ans.\n- Le meurtre commis sur un ascendant légitime ou naturel, sur les père ou mère adoptifs.\n- Le meurtre commis sur une personne dont la particulière vulnérabilité est apparente ou connue de l'auteur (due à son âge, à une maladie, à une infirmité, à une déficience physique ou psychique, ou à un état de grossesse).\n- Le meurtre commis sur un magistrat, un juré, un avocat, un officier public ou ministériel, un militaire de la gendarmerie nationale, un fonctionnaire de la police nationale, des douanes, de l'administration pénitentiaire, ou toute autre personne dépositaire de l'autorité publique, un sapeur-pompier ou un marin-pompier, un gardien assermenté d'immeubles ou de groupes d'immeubles, ou un agent exerçant pour le compte d'un bailleur des fonctions de gardiennage ou de surveillance des immeubles à usage d'habitation.\n- Le meurtre commis s

In [26]:
print(res)

Les conditions qui pourraient entraîner une condamnation à perpétuité sont les suivantes :

- Le meurtre commis sur un mineur de quinze ans.
- Le meurtre commis sur un ascendant légitime ou naturel, sur les père ou mère adoptifs.
- Le meurtre commis sur une personne dont la particulière vulnérabilité est apparente ou connue de l'auteur (due à son âge, à une maladie, à une infirmité, à une déficience physique ou psychique, ou à un état de grossesse).
- Le meurtre commis sur un magistrat, un juré, un avocat, un officier public ou ministériel, un militaire de la gendarmerie nationale, un fonctionnaire de la police nationale, des douanes, de l'administration pénitentiaire, ou toute autre personne dépositaire de l'autorité publique, un sapeur-pompier ou un marin-pompier, un gardien assermenté d'immeubles ou de groupes d'immeubles, ou un agent exerçant pour le compte d'un bailleur des fonctions de gardiennage ou de surveillance des immeubles à usage d'habitation.
- Le meurtre commis sur un e

## Avec les sources c'est mieux

In [27]:
from langchain_core.runnables import RunnableParallel

rag_chain_with_sources = RunnableParallel(question=RunnablePassthrough(), source=retriever ,answer=rag_chain)



In [28]:
rag_chain_with_sources.invoke("Quels sont les conditions qui pourraient entraîner une condamnation à perpétuité?")

{'question': 'Quels sont les conditions qui pourraient entraîner une condamnation à perpétuité?',
 'source': [Document(page_content="Article 132-8\n\nLorsqu'une personne physique, déjà condamnée définitivement pour un crime ou pour un délit puni de dix\nans d'emprisonnement par la loi, commet un crime, le maximum de la peine de la réclusion criminelle ou de\nla détention criminelle est la perpétuité si le maximum fixé par la loi pour ce crime est de vingt ou trente ans.\nLe maximum de la peine est porté à trente ans de réclusion criminelle ou de détention criminelle si le crime\nest puni de quinze ans.", metadata={'source': 'uploads/code_penal.pdf'}),
  Document(page_content="Article 132-18\n\nLorsqu'une infraction est punie de la réclusion criminelle ou de la détention criminelle à perpétuité, la\njuridiction peut prononcer une peine de réclusion criminelle ou de détention criminelle à temps, ou une peine\nd'emprisonnement qui ne peut être inférieure à deux ans.\n\nLorsqu'une infracti

In [29]:
import re

def article_number_parser(sources):

    regex = r"Article [A-Za-z]?\d+(-\d+)?(?=\n)"

    output = []
    for source in sources : 
        match = re.search(regex, source.page_content)
        output.append(match.group())
        
    return output

In [30]:
from langchain_core.runnables import RunnableParallel

rag_chain_with_articles_number = RunnableParallel(question=RunnablePassthrough(),source=(retriever | article_number_parser) ,answer=rag_chain)



In [31]:
rag_chain_with_articles_number.invoke("Quels sont les conditions qui pourraient entraîner une condamnation à perpétuité?")

{'question': 'Quels sont les conditions qui pourraient entraîner une condamnation à perpétuité?',
 'source': ['Article 132-8',
  'Article 132-18',
  'Article 132-35',
  'Article 221-4'],
 'answer': "Les conditions qui pourraient entraîner une condamnation à perpétuité sont les suivantes :\n\n- Le meurtre commis sur un mineur de quinze ans.\n- Le meurtre commis sur un ascendant légitime ou naturel, sur les père ou mère adoptifs.\n- Le meurtre commis sur une personne dont la particulière vulnérabilité est apparente ou connue de l'auteur (due à son âge, à une maladie, à une infirmité, à une déficience physique ou psychique, ou à un état de grossesse).\n- Le meurtre commis sur un magistrat, un juré, un avocat, un officier public ou ministériel, un militaire de la gendarmerie nationale, un fonctionnaire de la police nationale, des douanes, de l'administration pénitentiaire, ou toute autre personne dépositaire de l'autorité publique, un sapeur-pompier ou un marin-pompier, un gardien assermen